# 06 - Delta Table Writer (Gold Layer)

Transforms silver layer node/edge data to final gold format for Fabric Graph.

**Input**:
- `silver_nodes` - Node records with full metadata
- `silver_edges` - Edge records with full metadata

**Output**:
- `gold_nodes` - Simplified node format for graph import
- `gold_edges` - Simplified edge format for graph import

## Gold Table Schemas

**gold_nodes:**
| Column | Type | Description |
|--------|------|-------------|
| id | STRING | Unique node identifier |
| labels | ARRAY<STRING> | Node type labels |
| properties | MAP<STRING, STRING> | Property key-value pairs |

**gold_edges:**
| Column | Type | Description |
|--------|------|-------------|
| source_id | STRING | Source node ID |
| target_id | STRING | Target node ID |
| type | STRING | Edge type name |
| properties | MAP<STRING, STRING> | Edge properties (if any) |

In [ ]:
# Configuration
INPUT_NODES = "silver_nodes"
INPUT_EDGES = "silver_edges"
OUTPUT_NODES = "gold_nodes"
OUTPUT_EDGES = "gold_edges"

# Write mode: "overwrite" for full re-translation, "append" for incremental
WRITE_MODE = "overwrite"

# Partitioning: set to True to partition nodes by first label (improves query performance)
PARTITION_BY_LABEL = False

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, ArrayType, MapType

In [ ]:
# Load silver tables
df_silver_nodes = spark.table(INPUT_NODES)
df_silver_edges = spark.table(INPUT_EDGES)

print(f"Silver nodes: {df_silver_nodes.count()}")
print(f"Silver edges: {df_silver_edges.count()}")

In [ ]:
# Preview silver_nodes schema
print("Silver nodes schema:")
df_silver_nodes.printSchema()

In [ ]:
# Transform nodes to gold format
# Select only the columns needed for graph import
df_gold_nodes = df_silver_nodes.select(
    F.col("id"),
    F.col("labels"),
    F.coalesce(
        F.col("properties"),
        F.create_map().cast(MapType(StringType(), StringType()))
    ).alias("properties")
)

# Ensure labels is never null
df_gold_nodes = df_gold_nodes.withColumn(
    "labels",
    F.coalesce(F.col("labels"), F.array(F.lit("Entity")))
)

# Add primary_label for partitioning (first label in array)
if PARTITION_BY_LABEL:
    df_gold_nodes = df_gold_nodes.withColumn(
        "primary_label",
        F.col("labels")[0]
    )

print(f"Gold nodes prepared: {df_gold_nodes.count()}")

In [ ]:
# Preview gold nodes
print("\nGold nodes schema:")
df_gold_nodes.printSchema()

print("\nSample gold nodes:")
df_gold_nodes.show(10, truncate=50)

In [ ]:
# Preview silver_edges schema
print("Silver edges schema:")
df_silver_edges.printSchema()

In [ ]:
# Transform edges to gold format
df_gold_edges = df_silver_edges.select(
    F.col("source_id"),
    F.col("target_id"),
    F.col("type"),
    # Add empty properties map (edges don't have properties in current implementation)
    F.create_map().cast(MapType(StringType(), StringType())).alias("properties")
)

print(f"Gold edges prepared: {df_gold_edges.count()}")

In [ ]:
# Preview gold edges
print("\nGold edges schema:")
df_gold_edges.printSchema()

print("\nSample gold edges:")
df_gold_edges.show(10, truncate=40)

In [ ]:
# Validation: Check edge references
print("\n" + "=" * 60)
print("VALIDATION")
print("=" * 60)

node_ids = set(row.id for row in df_gold_nodes.select("id").distinct().collect())
edge_sources = set(row.source_id for row in df_gold_edges.select("source_id").distinct().collect())
edge_targets = set(row.target_id for row in df_gold_edges.select("target_id").distinct().collect())

missing_sources = edge_sources - node_ids
missing_targets = edge_targets - node_ids

if missing_sources:
    print(f"WARNING: {len(missing_sources)} edges reference missing source nodes")
else:
    print("[OK] All edge sources exist in nodes")

if missing_targets:
    print(f"WARNING: {len(missing_targets)} edges reference missing target nodes")
else:
    print("[OK] All edge targets exist in nodes")

# Check for null IDs
null_node_ids = df_gold_nodes.filter(F.col("id").isNull()).count()
if null_node_ids > 0:
    print(f"WARNING: {null_node_ids} nodes have null ID")
else:
    print("[OK] No null node IDs")

In [ ]:
# Summary statistics
print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)

node_count = df_gold_nodes.count()
edge_count = df_gold_edges.count()
unique_labels = df_gold_nodes.select(F.explode("labels").alias("label")).distinct().count()
unique_edge_types = df_gold_edges.select("type").distinct().count()

print(f"\nNodes: {node_count}")
print(f"  - Unique labels: {unique_labels}")
print(f"\nEdges: {edge_count}")
print(f"  - Unique edge types: {unique_edge_types}")
print(f"\nWrite mode: {WRITE_MODE}")
print(f"Partition by label: {PARTITION_BY_LABEL}")

In [ ]:
# Label distribution
print("\nNode label distribution:")
df_gold_nodes.select(F.explode("labels").alias("label")) \
    .groupBy("label").count() \
    .orderBy(F.desc("count")) \
    .show(20, truncate=40)

In [ ]:
# Edge type distribution
print("\nEdge type distribution:")
df_gold_edges.groupBy("type").count() \
    .orderBy(F.desc("count")) \
    .show(20, truncate=40)

In [ ]:
# Write gold_nodes to Delta table
print(f"\nWriting nodes to '{OUTPUT_NODES}' (mode={WRITE_MODE})...")

writer = df_gold_nodes.write \
    .format("delta") \
    .mode(WRITE_MODE) \
    .option("overwriteSchema", "true")

if PARTITION_BY_LABEL:
    writer = writer.partitionBy("primary_label")

writer.saveAsTable(OUTPUT_NODES)

print(f"Saved {node_count} nodes to '{OUTPUT_NODES}'")

In [ ]:
# Write gold_edges to Delta table
print(f"\nWriting edges to '{OUTPUT_EDGES}' (mode={WRITE_MODE})...")

df_gold_edges.write \
    .format("delta") \
    .mode(WRITE_MODE) \
    .option("overwriteSchema", "true") \
    .saveAsTable(OUTPUT_EDGES)

print(f"Saved {edge_count} edges to '{OUTPUT_EDGES}'")

In [ ]:
# Verify written tables
print("\n" + "=" * 60)
print("VERIFICATION")
print("=" * 60)

df_verify_nodes = spark.table(OUTPUT_NODES)
df_verify_edges = spark.table(OUTPUT_EDGES)

print(f"\nVerified {OUTPUT_NODES}: {df_verify_nodes.count()} rows")
print(f"Verified {OUTPUT_EDGES}: {df_verify_edges.count()} rows")

In [ ]:
print("\n" + "=" * 60)
print("DELTA TABLE WRITER COMPLETE")
print("=" * 60)
print(f"\nGold tables ready for Fabric Graph import:")
print(f"  - {OUTPUT_NODES}: {node_count} nodes")
print(f"  - {OUTPUT_EDGES}: {edge_count} edges")
print(f"\nNext: Run 07_graph_builder to generate Graph Model JSON")